[WordAndDoc2vec](https://github.com/darecophoenixx/wordroid.sblo.jp/wiki/WordAndDoc2vec)

In [ ]:
!pip install japanize-matplotlib

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

import japanize_matplotlib
plt.rcParams['font.family']

In [ ]:
img_cnt = 0

In [ ]:
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.png'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name)

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
df_part = pd.read_csv('../input/ks2020-encode-data-001/encoded_01.csv', index_col=0)
df_part.head()

In [ ]:
df_out_wgt_user = pd.read_csv('../input/ks2020-lowcol-fet12-001/out_user.csv', index_col=0)
X_df_user = df_out_wgt_user
wgt_user = df_out_wgt_user.values
df_out_wgt_user.head()

In [ ]:
df_out_wgt_col = pd.read_csv('../input/ks2020-lowcol-fet12-001/out_col.csv', index_col=0)
X_df_col = df_out_wgt_col
wgt_prod = df_out_wgt_col.values
df_out_wgt_col.head()

In [ ]:
X_df = df_out_wgt_user

In [ ]:
from som import som
from som.keras import sksom_keras2

K_shape = (4, 3)
sinit = som.SimpleSOM(K_shape, initialization_func='linear')
sinit._initialize(X_df.values)

In [ ]:
img = som.conv2img(sinit.K, K_shape)
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
K_init = sinit.K
sobj = sksom_keras2(kshape=K_shape, init_K=K_init, form='hex')

In [ ]:
df1= pd.DataFrame(sobj.landmarks_[:,:5])
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_df.values[:,:5])
df2['cls'] = 'X'
df2.head()
df = pd.concat([df2, df1], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5}, diag_kind='hist')
save_img()

In [ ]:
%%time
from keras.optimizers import RMSprop, Adam, SGD
hst = sobj.fit(X_df.values, batch_size=X_df.values.shape[0])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(list(range(len(sobj.hst['MeanDist2ClosestLM']))), sobj.hst['MeanDist2ClosestLM'], label="MeanDist2ClosestLM")
plt.legend(loc="best")
save_img()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(list(range(len(sobj.hst['loss']))), sobj.hst['loss'], label="Train loss")
plt.legend(loc="best")
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
ax = sobj.plot_hex(figsize=(3.6, 3.8), s=6900)

for i, m in enumerate(sobj.predict(X_df_col.values)):
    b, a = divmod(m, sobj.kshape[1])
    txt = X_df_col.index.values[i]
    plt.text(a, b, txt, ha='center', va='center',
           bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(20, 20))
plt.imshow(img)

for i, m in enumerate(sobj.predict(X_df_col.values)):
    b, a = divmod(m, sobj.kshape[1])
    txt = X_df_col.index.values[i]
    plt.text(a, b, txt, ha='center', va='center',
           bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))

save_img()

In [ ]:
sobj.predict(X_df.values)

In [ ]:
df1= pd.DataFrame(sobj.landmarks_[:,:5])
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_df.values[:,:5])
df2['cls'] = 'X'
df2.head()
df = pd.concat([df2, df1], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5}, diag_kind='hist')
save_img()

In [ ]:
%%time
hst = sobj.fit(X_df.values, r=(K_shape[0]/30.0, K_shape[0]/30.0), batch_size=X_df.values.shape[0], optimizer=Adam(learning_rate=0.001/8))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(list(range(len(sobj.hst['MeanDist2ClosestLM']))), sobj.hst['MeanDist2ClosestLM'], label="MeanDist2ClosestLM")
plt.legend(loc="best")
save_img()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(list(range(len(sobj.hst['loss']))), sobj.hst['loss'], label="Train loss")
plt.legend(loc="best")
save_img()

In [ ]:
sobj.landmarks_[:3,:3]

In [ ]:
sobj.kmeans.cluster_centers_[:3,:3]

In [ ]:
K_init[:3,:3]

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
ax = sobj.plot_hex(figsize=(3.6, 3.8), s=6900)

for i, m in enumerate(sobj.predict(X_df_col.values)):
    b, a = divmod(m, sobj.kshape[1])
    txt = X_df_col.index.values[i]
    plt.text(a, b, txt, ha='center', va='center',
           bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(20, 20))
plt.imshow(img)

for i, m in enumerate(sobj.predict(X_df_col.values)):
    b, a = divmod(m, sobj.kshape[1])
    txt = X_df_col.index.values[i]
    plt.text(a, b, txt, ha='center', va='center',
           bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
save_img()

In [ ]:
df1= pd.DataFrame(sobj.landmarks_[:,:5])
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_df.values[:,:5])
df2['cls'] = 'X'
df2.head()
df = pd.concat([df2, df1], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5}, diag_kind='hist')
save_img()

In [ ]:
ix, jx = 0,1
plt.figure(figsize=(10,10))
ax = sns.kdeplot(data=X_df_user, x=str(ix), y=str(jx),
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(sobj.landmarks_[:,ix], sobj.landmarks_[:,jx], marker='X')
for ii, col in enumerate(range(sobj.landmarks_.shape[0])):
        text = 'クラス' + str(ii)
        xy = (sobj.landmarks_[ii,ix], sobj.landmarks_[ii,jx])
        ax.annotate(text, xy=xy)
save_img()

In [ ]:
ix, jx = 0,2
plt.figure(figsize=(10,10))
ax = sns.kdeplot(data=X_df_user, x=str(ix), y=str(jx),
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(sobj.landmarks_[:,ix], sobj.landmarks_[:,jx], marker='X')
for ii, col in enumerate(range(sobj.landmarks_.shape[0])):
        text = 'クラス' + str(ii)
        xy = (sobj.landmarks_[ii,ix], sobj.landmarks_[ii,jx])
        ax.annotate(text, xy=xy)
save_img()

In [ ]:
df_res = pd.DataFrame({'cls': sobj.predict(X_df.values[:X_df_user.shape[0],:])}, index=X_df_user.index)
df_res

In [ ]:
cnt = pd.DataFrame({'grp': sobj.predict(X_df.values[:X_df_user.shape[0],:])}).groupby('grp').size()
cnt

In [ ]:
df_part.columns.values
ll = []
for ee in df_part.columns.values:
    tmp = pd.crosstab(df_res['cls'], df_part[ee], margins=True, normalize='index')
    #print(tmp)
    ll.append(tmp.values[:,1])

res = pd.DataFrame(np.c_[ll].T, index=tmp.index, columns=df_part.columns)
res.iloc[:,:10]

In [ ]:
res.iloc[:,10:20]

In [ ]:
res.iloc[:,20:30]

In [ ]:
res.iloc[:,30:]

In [ ]:
cnt1 = cnt.values
cnt1 = np.r_[cnt1, cnt.values.sum()]
cnt1 = pd.DataFrame(cnt1)
cnt1.index = res.index.values
cnt1
res1 = pd.concat([res, pd.DataFrame(cnt1)], axis=1)
res1.columns.values[-1] = 'cnt'
res1

In [ ]:
df_lm = pd.DataFrame(sobj.landmarks_)
df_lm

In [ ]:
X_df_col_som_out = X_df_col.copy()
X_df_col_som_out['cls'] = sobj.predict(X_df_col.values)
X_df_col_som_out

In [ ]:
X_df_user_som_out = X_df_user.copy()
X_df_user_som_out['cls'] = sobj.predict(X_df_user.values)
X_df_user_som_out

In [ ]:
X_df_user_som_out.to_csv('som_out_user.csv')
X_df_col_som_out.to_csv('som_out_col.csv')
df_lm.to_csv('som_out_LM.csv')
res1.to_csv('som_cls_crosstab.csv')